# Lekiwi

## Calibration

In [ ]:
!poetry run python -m lerobot.calibrate --teleop.type=so100_leader --teleop.port=/dev/tty.usbmodem5A460849101 --teleop.id=leader_s101

In [ ]:
!python examples/lekiwi/teleoperate.py

In [ ]:
!poetry show pyzmq

In [1]:
from lerobot.common.robots.lekiwi import LeKiwiClient, LeKiwiClientConfig
from lerobot.common.teleoperators.keyboard.teleop_keyboard import KeyboardTeleop, KeyboardTeleopConfig
from lerobot.common.teleoperators.so101_leader import SO101Leader, SO101LeaderConfig

In [2]:
import zmq
import time
from lerobot.common.errors import DeviceAlreadyConnectedError, DeviceNotConnectedError

class MyRobot(LeKiwiClient):
    def connect(self):  # Override the method
        """Establishes ZMQ sockets with the remote mobile robot"""

        if self._is_connected:
            raise DeviceAlreadyConnectedError(
                "LeKiwi Daemon is already connected. Do not run `robot.connect()` twice."
            )

        print(self.remote_ip)
        self.zmq_context = zmq.Context()
        self.zmq_cmd_socket = self.zmq_context.socket(zmq.PUSH)
        zmq_cmd_locator = f"tcp://{self.remote_ip}:{self.port_zmq_cmd}"
        self.zmq_cmd_socket.connect(zmq_cmd_locator)
        self.zmq_cmd_socket.setsockopt(zmq.CONFLATE, 1)

        self.zmq_observation_socket = self.zmq_context.socket(zmq.PULL)
        zmq_observations_locator = f"tcp://{self.remote_ip}:{self.port_zmq_observations}"
        self.zmq_observation_socket.connect(zmq_observations_locator)
        self.zmq_observation_socket.setsockopt(zmq.CONFLATE, 1)

        # Optional: give ZMQ time to finish the TCP handshake
        time.sleep(0.5)

        # 🔍 TEST CONNECTION: Blocking receive (only for debugging)
        print("Waiting for first message from host...")
        try:
            msg = self.zmq_observation_socket.recv_string()  # Blocking
            print("✅ Received first observation message:", msg[:200], "...")
        except Exception as e:
            print("❌ recv_string failed:", e)

        poller = zmq.Poller()
        poller.register(self.zmq_observation_socket, zmq.POLLIN)
        socks = dict(poller.poll(self.connect_timeout_s * 1000))
        print("socks:", socks)
        if self.zmq_observation_socket not in socks:
            print("zmq_observation_socket not in socks, not connected?")
            raise DeviceNotConnectedError("Timeout waiting for LeKiwi Host to connect expired.")
        if socks[self.zmq_observation_socket] != zmq.POLLIN:
            print("different from zmq.POLLIN, not connected?")
            raise DeviceNotConnectedError("Timeout waiting for LeKiwi Host to connect expired.")

        self._is_connected = True

In [5]:


robot_config = LeKiwiClientConfig(remote_ip="192.168.8.191", id="lekiwi_101")

teleop__arm_config = SO101LeaderConfig(
    port="/dev/tty.usbmodem5A460849101",
    id="leader_101",
)

teleop_keyboard_config = KeyboardTeleopConfig(
    id="mac_keyboard",
)

# robot = LeKiwiClient(robot_config)
robot = MyRobot(robot_config)
teleop_arm = SO101Leader(teleop__arm_config)
teleop_keyboard = KeyboardTeleop(teleop_keyboard_config)
teleop_keyboard.connect()
# robot.connect()
# teleop_arm.connect()


while True:
    # print(teleop_keyboard.get_action())
    action = teleop_keyboard.get_action()
    if ("x" in action):
        print(action)
        print("Moving forward")
        print(robot._from_keyboard_to_base_action({"x"}))
    # observation = robot.get_observation()

    # arm_action = teleop_arm.get_action()
    # arm_action = {f"arm_{k}": v for k, v in arm_action.items()}

    # keyboard_keys = telep_keyboard.get_action()
    # base_action = robot._from_keyboard_to_base_action(keyboard_keys)

    # robot.send_action(arm_action | base_action)


{'x': None}
Moving forward
{'x.vel': 0.0, 'y.vel': 0.0, 'theta.vel': -30.0}
{'x': None}
Moving forward
{'x.vel': 0.0, 'y.vel': 0.0, 'theta.vel': -30.0}
{'x': None}
Moving forward
{'x.vel': 0.0, 'y.vel': 0.0, 'theta.vel': -30.0}
{'x': None}
Moving forward
{'x.vel': 0.0, 'y.vel': 0.0, 'theta.vel': -30.0}
{'x': None}
Moving forward
{'x.vel': 0.0, 'y.vel': 0.0, 'theta.vel': -30.0}
{'x': None}
Moving forward
{'x.vel': 0.0, 'y.vel': 0.0, 'theta.vel': -30.0}
{'x': None}
Moving forward
{'x.vel': 0.0, 'y.vel': 0.0, 'theta.vel': -30.0}
{'x': None}
Moving forward
{'x.vel': 0.0, 'y.vel': 0.0, 'theta.vel': -30.0}
{'x': None}
Moving forward
{'x.vel': 0.0, 'y.vel': 0.0, 'theta.vel': -30.0}
{'x': None}
Moving forward
{'x.vel': 0.0, 'y.vel': 0.0, 'theta.vel': -30.0}
{'x': None}
Moving forward
{'x.vel': 0.0, 'y.vel': 0.0, 'theta.vel': -30.0}
{'x': None}
Moving forward
{'x.vel': 0.0, 'y.vel': 0.0, 'theta.vel': -30.0}
{'x': None}
Moving forward
{'x.vel': 0.0, 'y.vel': 0.0, 'theta.vel': -30.0}
{'x': None}


KeyboardInterrupt: 

In [ ]:
!python client.py